In [1]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import f1_score, roc_auc_score
import json

In [2]:
#X = pd.read_csv('./IU_Xray/clean_data.csv')

#Y = pd.read_csv('./IU_Xray/labeled_reports.csv')

#Y['report_id'] = X['uid'].apply(lambda x: 'CXR'+str(x))

#Y = Y[['report_id','Enlarged Cardiomediastinum', 'Cardiomegaly',
#       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
#       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
#       'Fracture', 'Support Devices', 'No Finding']]#.to_csv('chexbert_labels.csv')

#Y = Y.fillna(0)
#Y.replace(-1,1)
#Y.to_csv('./IU_Xray/chexbert_labels.csv')

In [3]:
labels_path = '/home/zaheer/pythonCode/MIMIC_CXR/'

In [4]:
# X = pd.read_csv(labels_path+'mimic_impression.csv')
X = pd.read_json(labels_path+'mimic_50k_all.json')


Y = pd.read_csv(labels_path+'labeled_reports.csv')

Y['report_id'] = [image[0] for image in X['image_path'].tolist()]

Y = Y[['report_id','Enlarged Cardiomediastinum', 'Cardiomegaly',
      'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
      'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
      'Fracture', 'Support Devices', 'No Finding']]#.to_csv('chexbert_labels.csv')

Y = Y.fillna(0)
Y = Y.replace(-1,1)
Y.to_csv(labels_path+'chexbert_labels.csv', index = False)

In [5]:
Y.head(6)

,report_id,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
0,p10/p10000032/s50414267/02aa804e-bde0afdd-112c...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,p10/p10000032/s50414267/174413ec-4ec4c1f7-34ea...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,p10/p10000032/s53189527/2a2277a9-b0ded155-c0de...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,p10/p10000032/s53189527/e084de3b-be89b11e-20fe...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,p10/p10000032/s53911762/68b5c4b1-227d0485-9cc3...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,p10/p10000032/s53911762/fffabebf-74fd3a1f-673b...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
path = labels_path

In [7]:
with open(path+'mimic_densenet_image_features.pkl','rb') as features:
#with open(path+'densenet_report_features.pkl','rb') as features:
    data=pickle.load(features)

In [8]:
feature_shape = 1024

In [9]:
# def clean_r2gen_annotations():
    
#     tags = pd.read_csv('./IU_Xray/chexbert_labels.csv')
    
#     reports = tags['report_id'].tolist()
    
#     #print(reports)
    
#     with open(path+'annotation.json', 'rb') as f:
#         full_records = json.load(f)

#     splits=['train','val','test']
    
#     new_images_records={}
    
#     for s in splits:
#         new_records=[]
#         records=full_records[s]
#         for r in records:
#             for image in r['image_path']:
#                 if image.split('_')[0] in reports: 
#                     new_records.append(r)
#                     break
                
#         new_images_records[s]=new_records
        
#     with open(path+'new_annotation.json', 'w') as f:
#         json.dump(new_images_records , f, ensure_ascii = False )
        
#     return new_images_records

#annot = clean_r2gen_annotations()

cols = ['Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
      'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
      'Fracture', 'Support Devices', 'No Finding']

def fetch_tags(image_id, tags):

    new_tags=[]
    image_ids =[]

    for image in image_id:
        tag = tags[tags.report_id == image][cols]
        
        if tag.shape[0] != 0:
            tag = tag.values.tolist()[0]
            image_ids.append(image)
            new_tags.append(tag)

    return image_ids, new_tags


def convert_r2gen_annotations():
    with open(path+'mimic_annotation_10000.json', 'rb') as f:
        full_records = json.load(f)

    splits=['train','val','test']
    
    new_images_records={}
    
    for s in splits:
        images=[]
        records=full_records[s]
        for r in records:
            images.append(r['image_path'][0])
                
        new_images_records[s]=images
    return new_images_records

convert_r2gen_annotations()

def load_preprocess_data():
    tags = pd.read_csv(labels_path+'chexbert_labels.csv')
    converted_records = convert_r2gen_annotations()
    x_train = converted_records['train']
    x_test = converted_records['test']
    x_val = converted_records['val']
    # print(x_train[:5])

    x_train, train_tags=fetch_tags(x_train,tags)
    #print(train_tags)
    #y_train = mlb.fit_transform([train_tags])
    y_train=np.array(train_tags)
    #print(y_train)

    #print(y_train.head())
    
    
    x_test, test_tags=fetch_tags(x_test,tags)
    y_test = np.array(test_tags)
    
    
    x_val, val_tags=fetch_tags(x_val,tags)
    y_val = np.array(val_tags)
    return x_train,y_train,x_test,y_test, x_val,y_val



In [10]:
def load_features(split):
    features = []#np.empty((0,7,7,1024))
    for image in split:
        features.append(data[image])
    features=np.array(features)
    features=np.reshape(features,(-1,49,feature_shape))
    
    return features

In [11]:
def class_model(n):
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=(49,feature_shape)))
    
    model.add(layers.GlobalAveragePooling1D())
    #model.add(layers.Dense(128,activation='relu'))
    #model.add(layers.LeakyReLU(alpha=0.05))
    #model.add(layers.Dense(64,activation='relu'))
    #model.add(layers.Dropout(0.4))
    model.add(layers.Dense(n, activation="sigmoid"))

    model.summary()
    
    return model



In [12]:
images_train,y_train,images_test,y_test, images_val,y_val=load_preprocess_data()

In [13]:

train_features=load_features(images_train)
test_features=load_features(images_test)
val_features=load_features(images_val)

In [14]:
y_train.shape

(10000, 14)

In [15]:
model=class_model(14)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling1d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 14)                14350     
Total params: 14,350
Trainable params: 14,350
Non-trainable params: 0
_________________________________________________________________


In [16]:
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()
])

model.fit(train_features, y_train, epochs=20, callbacks=[earlyStopping],validation_data=(val_features, y_val), batch_size=16)

test_tags=model.predict(test_features)
val_tags=model.predict(val_features)
train_tag = model.predict(train_features)

Train on 10000 samples, validate on 500 samples
Epoch 1/20
10000/10000 [==============================] - 3s 261us/sample - loss: 0.3723 - precision: 0.1538 - recall: 0.0017 - auc: 0.7373 - val_loss: 0.3538 - val_precision: 0.7778 - val_recall: 0.0069 - val_auc: 0.7796
Epoch 2/20
10000/10000 [==============================] - 2s 159us/sample - loss: 0.3422 - precision: 0.6837 - recall: 0.0340 - auc: 0.7934 - val_loss: 0.3459 - val_precision: 0.6357 - val_recall: 0.0804 - val_auc: 0.7895
Epoch 3/20
10000/10000 [==============================] - 2s 159us/sample - loss: 0.3350 - precision: 0.6369 - recall: 0.0802 - auc: 0.8027 - val_loss: 0.3420 - val_precision: 0.5876 - val_recall: 0.1118 - val_auc: 0.7941
Epoch 4/20
10000/10000 [==============================] - 2s 160us/sample - loss: 0.3309 - precision: 0.6191 - recall: 0.1057 - auc: 0.8084 - val_loss: 0.3395 - val_precision: 0.6277 - val_recall: 0.1157 - val_auc: 0.7981
Epoch 5/20
10000/10000 [==============================] - 2s 159

In [17]:
#test_tags=np.where(test_tags<0.50,0,1)
test_tags

array([[0.13042206, 0.14643073, 0.231128  , ..., 0.03926414, 0.633878  ,
        0.19422522],
       [0.3266785 , 0.38499603, 0.48312125, ..., 0.04222128, 0.45350447,
        0.01774138],
       [0.19514054, 0.46749637, 0.3327315 , ..., 0.02829   , 0.45712405,
        0.06438312],
       ...,
       [0.25921917, 0.44756886, 0.42429626, ..., 0.05960006, 0.37680298,
        0.02777189],
       [0.18263197, 0.23857147, 0.34747034, ..., 0.04213339, 0.44984728,
        0.11108777],
       [0.15461382, 0.16985184, 0.3149309 , ..., 0.05546287, 0.26723439,
        0.13149768]], dtype=float32)

In [18]:
#val_tags=np.where(val_tags<0.50,0,1)
val_tags

array([[0.11829931, 0.3594808 , 0.2357029 , ..., 0.01949322, 0.3711081 ,
        0.30966452],
       [0.19044548, 0.31982765, 0.17065188, ..., 0.0244073 , 0.04049382,
        0.24368957],
       [0.21338552, 0.23437458, 0.33639795, ..., 0.0691283 , 0.09513831,
        0.08788532],
       ...,
       [0.09161422, 0.09121519, 0.13626105, ..., 0.02665401, 0.07868797,
        0.35698158],
       [0.22575793, 0.13926736, 0.4228856 , ..., 0.1080727 , 0.04855758,
        0.06652284],
       [0.084205  , 0.05419081, 0.19661316, ..., 0.03686368, 0.1833309 ,
        0.35781315]], dtype=float32)

In [19]:
y_test_0=np.where(y_test==-1.0,0,y_test)

In [20]:
y_test_1=np.where(y_test==-1.0,1,y_test)

In [21]:
#val_tags=[i[0] for i in val_tags.tolist()]
#val_tags=pd.Series(val_tags)

In [22]:
#print(f1_score(y_test,test_tags))

In [23]:
#np.max(y_test_1[:,cidx])

In [24]:
for cidx in range(y_test.shape[1]):
    print(roc_auc_score(pd.Series(y_test_1[:,cidx]), pd.Series(test_tags[:,cidx])))

0.5287265347539319
0.7148802971931756
0.6545827515471203
0.5139556623931624
0.768851398094218
0.6434188034188034
0.5465869456900847
0.661452205882353
0.6997354497354498
0.7449576638206854
0.6656378600823045
0.5027377136752136
0.747750569971632
0.6921679319966257


In [25]:
# for cidx in range(y_test.shape[1]):
#     print(f1_score(pd.Series(y_test_0[:,cidx]), pd.Series(test_tags[:,cidx])))

In [26]:
m=tf.keras.metrics.Recall()
m.update_state(y_test,test_tags)
m.result().numpy()

0.18597786

In [27]:
import pickle

all_tags={}
all_reports = []
test_true = np.array(y_test)
for idx,image in enumerate(images_test):
    print(image)
    all_tags[image] = test_tags[idx,:]
    #all_tags[image] = test_tags[idx,:]
    #all_tags[image.split('_')[0]]=test_true[idx,:]
    all_reports.append(image.split('_')[0])

val_true = np.array(y_val)
for idx,image in enumerate(images_val):
    all_tags[image] = val_tags[idx,:]
    #all_tags[image] = val_tags[idx,:]
    #all_tags[image.split('_')[0]]=val_true[idx,:]
    all_reports.append(image.split('_')[0])
    
train_true=np.array(y_train)
for idx,image in enumerate(images_train):
    # all_tags[image.split('_')[0]]=train_true[idx,:]
    all_tags[image] = train_tag[idx,:]
    #all_tags[image] = train_tag[idx,:]
    all_reports.append(image.split('_')[0])


all_reports = list(set(all_reports))

#print(all_tags['CXR3614'])
#with open(path+'chexbert_chex_r2gen.pkl','wb') as file:
with open(path+'mimic_chexbert_dense.pkl','wb') as file:
    pickle.dump(all_tags, file, protocol=2)
    

p11/p11906222/s56779415/345c27ae-8dc96bd7-cd59fd7f-e18c90bc-71bf8122.jpg
p13/p13263843/s51718410/0844862c-b31ad664-cb39e0fe-f457cc37-02e1b4ae.jpg
p14/p14434800/s52682048/0d9ee316-000a9e0c-be78c74d-62923605-0315f8e4.jpg
p14/p14841168/s51745439/f66fce26-7c002d5f-2c12f63f-8dd12c3a-92ec73bf.jpg
p13/p13352405/s51233388/c2d94ada-21f141cb-17d5c7a3-f5807bbe-e83b679a.jpg
p19/p19159236/s58268220/166ed666-3cf27b16-96e71ab7-5c3cb2e9-2f2c90d1.jpg
p13/p13263843/s51718410/feeef719-f6a236d0-bd85a338-4d4729c9-d91fdf3d.jpg
p17/p17340686/s59672442/67486f3c-a4ef806f-47d7541c-c1f00d2e-9c2f09fe.jpg
p13/p13078497/s51153042/fd3bd9f2-a6369422-700296fc-3ec78cc2-f5884010.jpg
p16/p16553329/s50112134/7ddd8e36-8b7ad07a-2157c5f0-e30755e5-e0a8ad3f.jpg
p18/p18512911/s59232798/8f3afa87-cb2c2fec-210903d7-8faa6559-a7b6bf8e.jpg
p18/p18929056/s58958987/0d6db000-b7832a09-4e80e472-89242ef5-20701513.jpg
p19/p19800337/s56050160/00c3905c-c62bcac5-af952060-8b2bd330-cc4848ac.jpg
p15/p15032623/s56426120/69e36e8f-cfe80296-fba1f08a-

In [28]:
import numpy as np    
arr = np.empty((0,3), int)
print("Empty array:")
print(arr)
arr = np.append(arr, np.array([[10,20,30]]), axis=0)
arr = np.append(arr, np.array([[40,50,60]]), axis=0)
print("After adding two new arrays:")
print(arr)

Empty array:
[]
After adding two new arrays:
[[10 20 30]
 [40 50 60]]


In [29]:
B = 2*[[3,4]]
A = arr
print(B)
np.insert(A,A.shape[1],B,axis=1)

[[3, 4], [3, 4]]


array([[10, 20, 30,  3,  3],
       [40, 50, 60,  4,  4]])

In [30]:
!jupyter nbconvert --to script mimic_2d_classification_chexbert.ipynb

[NbConvertApp] Converting notebook mimic_2d_classification_chexbert.ipynb to script
[NbConvertApp] Writing 8310 bytes to mimic_2d_classification_chexbert.py
